In [1]:
import sys
sys.path.append("/users/munozr/mycodes")
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math
from data_analysis_codes.tools import Cstyle
from data_analysis_codes.tools import Amplitudes as A
plt.style.use(Cstyle.style1)
import math
from data_analysis_codes.tools import LCDM
LCDM = LCDM.evo()
from data_analysis_codes.tools import EdS
EdS = EdS.evo()

In [2]:
def dichotomy(y_wanted, function, lower_bound, upper_bound, tolerance):
    x_low = lower_bound
    x_upp = upper_bound
    x_mid = (x_low + x_upp) / 2
    y_low = function(x_low)
    y_upp = function(x_upp)
    y_mid = function(x_mid)
    while abs(y_wanted / y_mid - 1) > tolerance:
        if y_wanted > y_mid:
            y_low = y_mid
            x_low = x_mid
            x_mid = (x_low + x_upp) / 2
            y_mid = function(x_mid)
        else:
            y_upp = y_mid
            x_upp = x_mid
            x_mid = (x_low + x_upp) / 2
            y_mid = function(x_mid)
    return x_mid

def nbr_time_steps(tIN, tF, aTA, dx, a_function):
    ti = tIN
    nbr_steps = 0
    courant_fac = 0.1
    while ti<tF and nbr_steps<4e6:
        dtv = courant_fac*np.sqrt(3)*a_function(ti)*dx
        dtc = courant_fac*np.sqrt(3)*aTA*dx
        if dtv<dtc:
            ti += dtv
        else:
            ti += dtc
        nbr_steps += 1
    return nbr_steps

# LCDM

In [3]:
# INI SIM 
# L = 4/aHIN
tIN = 1
aIN = LCDM.a(tIN)
zIN = LCDM.z(tIN)
L = [1821]*3
Amp_pert = [0.01138486133517004756]*3
output = A.f_Amp(tIN, L, Amp_pert)
nonlin = abs(output['deltaOD']/output['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output['deltaOD']), 'nonlin = {:.2e}'.format(nonlin))
print('Amp_pert = ', Amp_pert[0])

deltaTA = 1.06241
zTA = (1+zIN)*output['deltaOD']/deltaTA - 1
deltaC = 1.686
zC = (1+zIN)*output['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN), '{:.2f}'.format(zTA), '{:.2f}'.format(zC))
aTA = aIN*(deltaTA/output['deltaOD'])
aC = aIN*(deltaC/output['deltaOD'])
tTA = dichotomy(aTA, LCDM.a, tIN, 500, 1e-1)
tC = dichotomy(aC, LCDM.a, tTA, 500, 1e-1)
print('t = {:.2f}'.format(tIN), '{:.2f}'.format(tTA), '{:.2f}'.format(tC))

print('L = ', L[0], L[0]/2, L[0]/128, L[0]/64, L[0]/32)

dtfacprev = 0.0006 
print('dtfac = ', dtfacprev)
dx = L[0] / 128
dt = dtfacprev * dx
print('timesteps = ', (tC - tIN)/dt, nbr_time_steps(tIN, tC, aTA, dx, LCDM.a))

deltaOD = 3.00000000e-02 nonlin = 1.10e-01
Amp_pert =  0.011384861335170048
z = 302.53 7.57 4.40
t = 1.00 188.12 422.03
L =  1821 910.5 14.2265625 28.453125 56.90625
dtfac =  0.0006
timesteps =  49324.54695222406 2567


In [4]:
# L = 1/aHIN
tIN = 1
aIN = LCDM.a(tIN)
zIN = LCDM.z(tIN)
L = [455.3]*3 #[1/(aIN*LCDM.Hprop(tIN))]*3

deltaOD_wanted = 0.03
def delta(Amp):
    return A.f_Amp(tIN, L, [Amp]*3)['deltaOD']
Amp_pert = [dichotomy(deltaOD_wanted, delta, 1e-10, 0.1, 1e-8)]*3

output = A.f_Amp(tIN, L, Amp_pert)
nonlin = abs(output['deltaOD']/output['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output['deltaOD']), 'nonlin = {:.2e}'.format(nonlin))
print('Amp_pert = ', Amp_pert[0])

deltaTA = 1.06241
zTA = (1+zIN)*output['deltaOD']/deltaTA - 1
deltaC = 1.686
zC = (1+zIN)*output['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN), '{:.2f}'.format(zTA), '{:.2f}'.format(zC))
aTA = aIN*(deltaTA/output['deltaOD'])
aC = aIN*(deltaC/output['deltaOD'])
tTA = dichotomy(aTA, LCDM.a, tIN, 500, 1e-3)
tC = dichotomy(aC, LCDM.a, tTA, 500, 1e-3)
print('t = {:.2f}'.format(tIN), '{:.2f}'.format(tTA), '{:.2f}'.format(tC))

print('L = ', L[0], L[0]/2, L[0]/128, L[0]/64, L[0]/32)
print('--------------------------------')

tIN2 = 2.9
aIN2 = LCDM.a(tIN2)
zIN2 = LCDM.z(tIN2)
delta1OD_wanted2 = deltaOD_wanted * aIN2 / aIN
def delta12(Amp):
    return A.f_Amp(tIN2, L, [Amp]*3)['deltaOD']
Amp_pert2 = [dichotomy(delta1OD_wanted2, delta12, 1e-10, 10, 1e-8)]*3
output2 = A.f_Amp(tIN2, L, Amp_pert2)
nonlin2 = abs(output2['deltaOD']/output2['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output2['deltaOD']), 
      'delta1OD = {:.8e}'.format(output2['delta1OD']), 
      'nonlin = {:.2e}'.format(nonlin2))
print('Amp_pert = ', Amp_pert2[0])

zTA2 = (1+zIN2)*output2['deltaOD']/deltaTA - 1
zC2 = (1+zIN2)*output2['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN2), '{:.2f}'.format(zTA2), '{:.2f}'.format(zC2))
aTA2 = aIN2*(deltaTA/output2['deltaOD'])
aC2 = aIN2*(deltaC/output2['deltaOD'])
tTA2 = dichotomy(aTA2, LCDM.a, tIN2, 500, 1e-3)
tC2 = dichotomy(aC2, LCDM.a, tTA2, 500, 1e-3)
print('t = {:.2f}'.format(tIN2), '{:.2f}'.format(tTA2), '{:.2f}'.format(tC2))

dtfacprev = 0.0006 
dtfac = (dtfacprev / LCDM.a(tIN)) * LCDM.a(tIN2)
print('dtfac = ', dtfac)
dx = L[0] / 128
dt = dtfac * dx
print('timesteps = ', (tC2 - tIN2)/dt, nbr_time_steps(tIN2, tC2, aTA2, dx, LCDM.a))

deltaOD = 3.00000000e-02 nonlin = 4.57e-02
Amp_pert =  0.0006055992159283955
z = 302.53 7.57 4.40
t = 1.00 210.54 420.85
L =  455.3 227.65 3.55703125 7.1140625 14.228125
--------------------------------
deltaOD = 6.10079757e-02 delta1OD = 5.54117398e-02 nonlin = 1.01e-01
Amp_pert =  0.0005751665957585271
z = 148.26 7.57 4.40
t = 2.90 210.67 420.89
dtfac =  0.0012201595127336754
timesteps =  96307.12749513837 9613


In [5]:
# L = 1/4aHIN
tIN = 1
aIN = LCDM.a(tIN)
zIN = LCDM.z(tIN)
L = [113.825]*3 #[1/(4*aIN*LCDM.Hprop(tIN))]*3

deltaOD_wanted = 0.03
def delta(Amp):
    return A.f_Amp(tIN, L, [Amp]*3)['deltaOD']
Amp_pert = [dichotomy(deltaOD_wanted, delta, 1e-10, 0.1, 1e-8)]*3

output = A.f_Amp(tIN, L, Amp_pert)
nonlin = abs(output['deltaOD']/output['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output['deltaOD']), 'nonlin = {:.2e}'.format(nonlin))
print('Amp_pert = ', Amp_pert[0])

deltaTA = 1.06241
zTA = (1+zIN)*output['deltaOD']/deltaTA - 1
deltaC = 1.686
zC = (1+zIN)*output['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN), '{:.2f}'.format(zTA), '{:.2f}'.format(zC))
aTA = aIN*(deltaTA/output['deltaOD'])
aC = aIN*(deltaC/output['deltaOD'])
tTA = dichotomy(aTA, LCDM.a, tIN, 500, 1e-3)
tC = dichotomy(aC, LCDM.a, tTA, 500, 1e-3)
print('t = {:.2f}'.format(tIN), '{:.2f}'.format(tTA), '{:.2f}'.format(tC))

print('L = ', L[0], L[0]/2, L[0]/128, L[0]/64, L[0]/32)
print('--------------------------------')

tIN2 = 2.85
aIN2 = LCDM.a(tIN2)
zIN2 = LCDM.z(tIN2)
delta1OD_wanted2 = deltaOD_wanted * aIN2 / aIN
def delta12(Amp):
    return A.f_Amp(tIN2, L, [Amp]*3)['deltaOD']
Amp_pert2 = [dichotomy(delta1OD_wanted2, delta12, 1e-10, 10, 1e-8)]*3
output2 = A.f_Amp(tIN2, L, Amp_pert2)
nonlin2 = abs(output2['deltaOD']/output2['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output2['deltaOD']), 
      'delta1OD = {:.8e}'.format(output2['delta1OD']), 
      'nonlin = {:.2e}'.format(nonlin2))
print('Amp_pert = ', Amp_pert2[0])

zTA2 = (1+zIN2)*output2['deltaOD']/deltaTA - 1
zC2 = (1+zIN2)*output2['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN2), '{:.2f}'.format(zTA2), '{:.2f}'.format(zC2))
aTA2 = aIN2*(deltaTA/output2['deltaOD'])
aC2 = aIN2*(deltaC/output2['deltaOD'])
tTA2 = dichotomy(aTA2, LCDM.a, tIN2, 500, 1e-3)
tC2 = dichotomy(aC2, LCDM.a, tTA2, 500, 1e-3)
print('t = {:.2f}'.format(tIN2), '{:.2f}'.format(tTA2), '{:.2f}'.format(tC2))

dtfacprev = 0.0006 
dtfac = (dtfacprev / LCDM.a(tIN)) * LCDM.a(tIN2)
print('dtfac = ', dtfac)
dx = L[0] / 128
dt = dtfac * dx
print('timesteps = ', (tC2 - tIN2)/dt, nbr_time_steps(tIN2, tC2, aTA2, dx, LCDM.a))

deltaOD = 3.00000000e-02 nonlin = 5.48e-02
Amp_pert =  3.752201110720033e-05
z = 302.53 7.57 4.40
t = 1.00 210.54 420.85
L =  113.825 56.9125 0.8892578125 1.778515625 3.55703125
--------------------------------
deltaOD = 6.03047038e-02 delta1OD = 5.43986827e-02 nonlin = 1.09e-01
Amp_pert =  3.5702258811188794e-05
z = 150.00 7.57 4.40
t = 2.85 210.64 420.88
dtfac =  0.001206094074571842
timesteps =  389760.2344433602 38494


# EdS

In [13]:
# INI SIM 
# L = 4/aHIN
tIN = 1
aIN = EdS.a(tIN)
zIN = EdS.z(tIN)
L = [1206]*3
Amp_pert = [0.01068203445570103244]*3
output = A.f_Amp(tIN, L, Amp_pert, expansion='EdS')
nonlin = abs(output['deltaOD']/output['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output['deltaOD']), 'nonlin = {:.2e}'.format(nonlin))
print('Amp_pert = ', Amp_pert[0])

deltaTA = 1.06241
zTA = (1+zIN)*output['deltaOD']/deltaTA - 1
deltaC = 1.686
zC = (1+zIN)*output['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN), '{:.2f}'.format(zTA), '{:.2f}'.format(zC))
aTA = aIN*(deltaTA/output['deltaOD'])
aC = aIN*(deltaC/output['deltaOD'])
tTA = dichotomy(aTA, EdS.a, tIN, 500, 1e-3)
tC = dichotomy(aC, EdS.a, tTA, 500, 1e-3)
print('t = {:.2f}'.format(tIN), '{:.2f}'.format(tTA), '{:.2f}'.format(tC))

print('L = ', L[0], L[0]/2, L[0]/128, L[0]/64, L[0]/32)

dtfacprev = 0.0006 
print('dtfac = ', dtfacprev)
dx = L[0] / 128
dt = dtfacprev * dx
print('timesteps = ', (tC - tIN)/dt, nbr_time_steps(tIN, tC, aTA, dx, EdS.a))

deltaOD = 3.00000000e-02 nonlin = 1.01e-01
Amp_pert =  0.010682034455701032
z = 205.46 4.83 2.67
t = 1.00 210.54 420.85
L =  1206 603.0 9.421875 18.84375 37.6875
dtfac =  0.0006
timesteps =  74268.84403935185 2632


In [6]:
# L = 1/aHIN
tIN = 1
aIN = EdS.a(tIN)
zIN = EdS.z(tIN)
L = [310]*3 #[1/(aIN*EdS.Hprop(tIN))]*3

deltaOD_wanted = 0.03
def delta(Amp):
    return A.f_Amp(tIN, L, [Amp]*3, expansion='EdS')['deltaOD']
Amp_pert = [dichotomy(deltaOD_wanted, delta, 1e-10, 0.1, 1e-8)]*3

output = A.f_Amp(tIN, L, Amp_pert, expansion='EdS')
nonlin = abs(output['deltaOD']/output['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output['deltaOD']), 'nonlin = {:.2e}'.format(nonlin))
print('Amp_pert = ', Amp_pert[0])

deltaTA = 1.06241
zTA = (1+zIN)*output['deltaOD']/deltaTA - 1
deltaC = 1.686
zC = (1+zIN)*output['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN), '{:.2f}'.format(zTA), '{:.2f}'.format(zC))
aTA = aIN*(deltaTA/output['deltaOD'])
aC = aIN*(deltaC/output['deltaOD'])
tTA = dichotomy(aTA, EdS.a, tIN, 500, 1e-3)
tC = dichotomy(aC, EdS.a, tTA, 500, 1e-3)
print('t = {:.2f}'.format(tIN), '{:.2f}'.format(tTA), '{:.2f}'.format(tC))

print('L = ', L[0], L[0]/2, L[0]/128, L[0]/64, L[0]/32)
print('--------------------------------')

tIN2 = 2.9
aIN2 = EdS.a(tIN2)
zIN2 = EdS.z(tIN2)
delta1OD_wanted2 = deltaOD_wanted * aIN2 / aIN
def delta12(Amp):
    return A.f_Amp(tIN2, L, [Amp]*3, expansion='EdS')['deltaOD']
Amp_pert2 = [dichotomy(delta1OD_wanted2, delta12, 1e-10, 10, 1e-8)]*3
output2 = A.f_Amp(tIN2, L, Amp_pert2, expansion='EdS')
nonlin2 = abs(output2['deltaOD']/output2['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output2['deltaOD']), 'delta1OD = {:.8e}'.format(output2['delta1OD']), 'nonlin = {:.2e}'.format(nonlin2))
print('Amp_pert = ', Amp_pert2[0])

zTA2 = (1+zIN2)*output2['deltaOD']/deltaTA - 1
zC2 = (1+zIN2)*output2['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN2), '{:.2f}'.format(zTA2), '{:.2f}'.format(zC2))
aTA2 = aIN2*(deltaTA/output2['deltaOD'])
aC2 = aIN2*(deltaC/output2['deltaOD'])
tTA2 = dichotomy(aTA2, EdS.a, tIN2, 500, 1e-3)
tC2 = dichotomy(aC2, EdS.a, tTA2, 500, 1e-3)
print('t = {:.2f}'.format(tIN2), '{:.2f}'.format(tTA2), '{:.2f}'.format(tC2))

dtfacprev = 0.0006 
dtfac = (dtfacprev / EdS.a(tIN)) * EdS.a(tIN2)
print('dtfac = ', dtfac)
dx = L[0] / 128
dt = dtfac * dx
print('timesteps = ', (tC2 - tIN2)/dt, nbr_time_steps(tIN2, tC2, aTA2, dx, EdS.a))

deltaOD = 3.00000003e-02 nonlin = 4.56e-02
Amp_pert =  0.0006068159423047852
z = 205.46 4.83 2.67
t = 1.00 210.54 420.85
L =  310 155.0 2.421875 4.84375 9.6875
--------------------------------
deltaOD = 6.10079719e-02 delta1OD = 5.54126799e-02 nonlin = 1.01e-01
Amp_pert =  0.0005763211992824164
z = 100.53 4.83 2.67
t = 2.90 210.67 420.89
dtfac =  0.0012201594345070952
timesteps =  141447.21922500024 9604


In [7]:
# L = 1/aHIN
tIN = 1
aIN = EdS.a(tIN)
zIN = EdS.z(tIN)
L = [77]*3 #[1/(4*aIN*EdS.Hprop(tIN))]*3

deltaOD_wanted = 0.03
def delta(Amp):
    return A.f_Amp(tIN, L, [Amp]*3, expansion='EdS')['deltaOD']
Amp_pert = [dichotomy(deltaOD_wanted, delta, 1e-10, 0.1, 1e-8)]*3

output = A.f_Amp(tIN, L, Amp_pert, expansion='EdS')
nonlin = abs(output['deltaOD']/output['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output['deltaOD']), 'nonlin = {:.2e}'.format(nonlin))
print('Amp_pert = ', Amp_pert[0])

deltaTA = 1.06241
zTA = (1+zIN)*output['deltaOD']/deltaTA - 1
deltaC = 1.686
zC = (1+zIN)*output['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN), '{:.2f}'.format(zTA), '{:.2f}'.format(zC))
aTA = aIN*(deltaTA/output['deltaOD'])
aC = aIN*(deltaC/output['deltaOD'])
tTA = dichotomy(aTA, EdS.a, tIN, 500, 1e-3)
tC = dichotomy(aC, EdS.a, tTA, 500, 1e-3)
print('t = {:.2f}'.format(tIN), '{:.2f}'.format(tTA), '{:.2f}'.format(tC))

print('L = ', L[0], L[0]/2, L[0]/128, L[0]/64, L[0]/32)
print('--------------------------------')

tIN2 = 2.9
aIN2 = EdS.a(tIN2)
zIN2 = EdS.z(tIN2)
delta1OD_wanted2 = deltaOD_wanted * aIN2 / aIN
def delta12(Amp):
    return A.f_Amp(tIN2, L, [Amp]*3, expansion='EdS')['deltaOD']
Amp_pert2 = [dichotomy(delta1OD_wanted2, delta12, 1e-10, 10, 1e-8)]*3
output2 = A.f_Amp(tIN2, L, Amp_pert2, expansion='EdS')
nonlin2 = abs(output2['deltaOD']/output2['delta1OD'] - 1)
print('deltaOD = {:.8e}'.format(output2['deltaOD']), 'delta1OD = {:.8e}'.format(output2['delta1OD']), 'nonlin = {:.2e}'.format(nonlin2))
print('Amp_pert = ', Amp_pert2[0])

zTA2 = (1+zIN2)*output2['deltaOD']/deltaTA - 1
zC2 = (1+zIN2)*output2['deltaOD']/deltaC - 1
print('z = {:.2f}'.format(zIN2), '{:.2f}'.format(zTA2), '{:.2f}'.format(zC2))
aTA2 = aIN2*(deltaTA/output2['deltaOD'])
aC2 = aIN2*(deltaC/output2['deltaOD'])
tTA2 = dichotomy(aTA2, EdS.a, tIN2, 500, 1e-3)
tC2 = dichotomy(aC2, EdS.a, tTA2, 500, 1e-3)
print('t = {:.2f}'.format(tIN2), '{:.2f}'.format(tTA2), '{:.2f}'.format(tC2))

dtfacprev = 0.0006 
dtfac = (dtfacprev / EdS.a(tIN)) * EdS.a(tIN2)
print('dtfac = ', dtfac)
dx = L[0] / 128
dt = dtfac * dx
print('timesteps = ', (tC2 - tIN2)/dt, nbr_time_steps(tIN2, tC2, aTA2, dx, EdS.a))

deltaOD = 3.00000002e-02 nonlin = 5.48e-02
Amp_pert =  3.7112910932007065e-05
z = 205.46 4.83 2.67
t = 1.00 210.54 420.85
L =  77 38.5 0.6015625 1.203125 2.40625
--------------------------------
deltaOD = 6.10079714e-02 delta1OD = 5.49720537e-02 nonlin = 1.10e-01
Amp_pert =  3.527406131595393e-05
z = 100.53 4.83 2.67
t = 2.90 210.67 420.89
dtfac =  0.0012201594345070952
timesteps =  569462.8306461049 38659
